In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import catboost
import pyltr
import pickle

%matplotlib inline

In [2]:
import keras
from keras.models import load_model
from keras.losses import binary_crossentropy, mean_squared_error, mean_squared_error

/Users/sergmiller/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/sergmiller/anaconda3/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
import building, training, models, callbacks, wonder, generator, ranking

In [4]:
from importlib import reload

In [5]:
reload(building)
reload(training)
reload(models)
reload(callbacks)
reload(wonder)
reload(generator)
reload(ranking)

<module 'ranking' from '/Users/sergmiller/Documents/code/python/alissa/ranking.py'>

In [6]:
emb = building.make_emb('datasets/ru.csv')
train = pd.read_csv('datasets/sents/train_sents').fillna('')
val = pd.read_csv('datasets/sents/val_sents').fillna('')
boost = pd.read_csv('datasets/sents/boost_sents').fillna('')
test = pd.read_csv('datasets/sents/test_sents').fillna('')
public = pd.read_csv('datasets/public_df', index_col='Unnamed: 0').fillna('')
private = pd.read_csv('datasets/private_df', index_col='Unnamed: 0').fillna('')

In [7]:
train_val = pd.concat([train, val], axis=0)

In [8]:
train_val.shape

(78757, 8)

In [9]:
model = load_model('weights/golden/weights.02-0.985.hdf5')

model.compile(
    loss={'class_out': binary_crossentropy, 
          'auto3' : mean_squared_error, 
          'auto4': mean_squared_error},
    loss_weights = {'class_out': 1, 
          'auto3' : 0, 
          'auto4': 0},
    optimizer='rmsprop',
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


must load end!

In [9]:
train.head()

,0,1,2,3,4,5,6,7
0,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,0,"да , в ванной .",good,0.942226
1,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,1,я не грязный .,bad,0.812665
2,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,2,"так , будь готова .",good,0.488691
3,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,3,на тот холм .,bad,0.996344
4,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,4,ты идешь внутрь .,bad,0.506997


In [10]:
public.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [63]:
boost.shape

(13899, 8)

In [206]:
def run_model(train, val, emb, epochs=20, batch_size=64, sample_len=10000, load_file=None, save_file=None):
    if load_file is None:
        model = models.model((40, 300), eps=1e-5)
    else:
        model = load_model(load_file)

        model.compile(
            loss={'class_out': binary_crossentropy, 
                  'auto3' : mean_squared_error, 
                  'auto4': mean_squared_error},
            loss_weights = {'class_out': 1, 
                  'auto3' : 0, 
                  'auto4': 0},
            optimizer='rmsprop',
        )
    
    filepath = "weights/weights.{epoch:02d}-{val_loss:.3f}.hdf5"
    
    y_train = (train['6'] == 'good').values
    y_val = (val['6'] == 'good').values
    w_train = train['7']
    
    sample_train = building.shuffle_by_groups(train, '0', random_state=0)[:sample_len]

    model.fit_generator(training.flow(train, emb, batch_size, y_train, w_train),
                        steps_per_epoch=train.shape[0] / batch_size, 
                        epochs=epochs, 
                        shuffle=True,
                        callbacks=[callbacks.TelegramCallback(), 
                                callbacks.RocCallback(sample_train, val, emb),
                                callbacks.NDCGCallback(sample_train, val, emb),
                                keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=3, min_lr=1e-5),
                                keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                                    save_best_only=False, save_weights_only=False, mode='auto', period=1)
                                  ],
                        validation_data=training.flow(val, emb, batch_size, y_train),
                        validation_steps=val.shape[0] / batch_size,
                       )

    if save_file is not None:
        model.save(save_file)

In [207]:
run_model(train, val, emb, load_file='weights/weights.06-2.292.hdf5')

Epoch 1/20
1108/1107 [==============================] - 339s 306ms/step - loss: 0.5368 - class_out_loss: 0.5137 - auto3_loss: 0.9561 - auto4_loss: 0.8353 - val_loss: 0.9785 - val_class_out_loss: 0.9547 - val_auto3_loss: 0.9770 - val_auto4_loss: 0.8560
Epoch 2/20
1108/1107 [==============================] - 302s 273ms/step - loss: 0.5136 - class_out_loss: 0.4884 - auto3_loss: 0.9654 - auto4_loss: 0.8447 - val_loss: 0.9847 - val_class_out_loss: 0.9581 - val_auto3_loss: 0.9757 - val_auto4_loss: 0.8546
Epoch 3/20
1108/1107 [==============================] - 251s 227ms/step - loss: 4.7504 - class_out_loss: 4.7208 - auto3_loss: 0.9655 - auto4_loss: 0.8446 - val_loss: 8.5274 - val_class_out_loss: 8.5023 - val_auto3_loss: 0.9771 - val_auto4_loss: 0.8553
Epoch 4/20
1108/1107 [==============================] - 304s 274ms/step - loss: 6.3211 - class_out_loss: 6.2982 - auto3_loss: 0.9655 - auto4_loss: 0.8447 - val_loss: 7.7430 - val_class_out_loss: 7.7206 - val_auto3_loss: 0.9776 - val_auto4_loss:

In [10]:
training.predict_with_nn(model, test, emb, use_y=True)

ndcg_mean: 0.9033897938335989, ndcg_std: 0.14468554771085015


array([[0.6410377 ],
       [0.97201127],
       [0.58125263],
       ...,
       [0.61131245],
       [0.74294466],
       [0.7219546 ]], dtype=float32)

у недоученной модели(автоэнкодер) 0.892 +- 0.150, лидерборд - 0.84742

у четкой модели - 0.903 +- 0.145, лидерборд - 0.84497

In [15]:
reload(training)

<module 'training' from '/Users/sergmiller/Documents/code/python/alissa/training.py'>

In [14]:
training.predict_with_nn(model, boost, emb, use_y=True)

[[0.4160223 ]
 [0.35829148]
 [0.3237929 ]
 ...
 [0.29188445]
 [0.8074343 ]
 [0.60766774]]
ndcg_mean: 0.8998286971054842, ndcg_std: 0.14367885667958052


array([[0.4160223 ],
       [0.35829148],
       [0.3237929 ],
       ...,
       [0.29188445],
       [0.8074343 ],
       [0.60766774]], dtype=float32)

In [98]:
public_pred = training.predict_with_nn(model, public, emb, use_y=False)

In [99]:
sub = wonder.create_sub(public, public_pred, 'submissions/sub_only_with_tuned_model0.csv')

## pyltr

In [9]:
with open('model_pyltr', 'rb') as f:
    rank_model = pickle.load(f)[0]

In [38]:
metric = pyltr.metrics.NDCG(k=6, gain_type='identity')

rank_model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=1000,
    learning_rate=0.05,
    max_features=0.3,
    subsample=1,
    random_state=1,
    max_leaf_nodes=8,
    min_samples_leaf=100,
    verbose=1,
)

In [40]:
res, rank_model = generator.train_pyltr(rank_model, metric, \
                                   model, emb, boost, \
                                   model_file='rank_model', use_cache=True, stop_after=500)

In [33]:
res

(0.9172111600057461, 0.1250853174634983)

In [34]:
y_pred = training.predict_with_stack_model(rank_model, model, test, emb, use_y=True)

/Users/sergmiller/Documents/code/python/alissa/generator.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  output=model.get_layer(model.layers[5].name).output)


ndcg_mean: 0.9017489184385077, ndcg_std: 0.14364126367230878


## catboost

In [10]:
rank_model = catboost.CatBoostRegressor(
                            iterations=4000, 
                            depth=7, 
                            rsm=0.1, 
                            l2_leaf_reg=1e-2,
                            thread_count=4,
                            learning_rate= 1e-1,
                            loss_function='YetiRank',
                            random_state=1,
                            bagging_temperature=0.01,
                            use_best_model=True,
                                  )

In [12]:
reload(generator)

<module 'generator' from '/Users/sergmiller/Documents/code/python/alissa/generator.py'>

In [13]:
reload(catboost)

<module 'catboost' from '/Users/sergmiller/anaconda3/envs/py35/lib/python3.5/site-packages/catboost/__init__.py'>

In [14]:
type(rank_model)

catboost.core.CatBoostRegressor

In [16]:
res, rank_model = generator.train_catboost(rank_model, \
                                model, emb, train, boost, \
                                model_file='rank_model_cat', use_cache=True)

Datasets are loaded
0:	learn: 0.8991853	test: 0.8993028	best: 0.8993028 (0)	total: 335ms	remaining: 22m 19s
1:	learn: 0.9139524	test: 0.9130353	best: 0.9130353 (1)	total: 1.64s	remaining: 54m 35s
2:	learn: 0.9165840	test: 0.9160364	best: 0.9160364 (2)	total: 1.89s	remaining: 42m 1s
3:	learn: 0.9170976	test: 0.9170494	best: 0.9170494 (3)	total: 2.36s	remaining: 39m 18s
4:	learn: 0.9174416	test: 0.9178708	best: 0.9178708 (4)	total: 2.71s	remaining: 36m 4s
5:	learn: 0.9174507	test: 0.9173025	best: 0.9178708 (4)	total: 3.15s	remaining: 34m 58s
6:	learn: 0.9172404	test: 0.9179139	best: 0.9179139 (6)	total: 3.44s	remaining: 32m 43s
7:	learn: 0.9174856	test: 0.9181192	best: 0.9181192 (7)	total: 4.3s	remaining: 35m 47s
8:	learn: 0.9174754	test: 0.9185586	best: 0.9185586 (8)	total: 4.57s	remaining: 33m 47s
9:	learn: 0.9175241	test: 0.9179879	best: 0.9185586 (8)	total: 5.11s	remaining: 33m 59s
10:	learn: 0.9179832	test: 0.9184005	best: 0.9185586 (8)	total: 5.32s	remaining: 32m 9s
11:	learn: 0.91

92:	learn: 0.9203036	test: 0.9188199	best: 0.9189860 (21)	total: 29.1s	remaining: 20m 24s
93:	learn: 0.9203006	test: 0.9188765	best: 0.9189860 (21)	total: 29.3s	remaining: 20m 19s
94:	learn: 0.9203326	test: 0.9188399	best: 0.9189860 (21)	total: 29.5s	remaining: 20m 14s
95:	learn: 0.9203392	test: 0.9188359	best: 0.9189860 (21)	total: 29.7s	remaining: 20m 9s
96:	learn: 0.9203435	test: 0.9187014	best: 0.9189860 (21)	total: 29.9s	remaining: 20m 4s
97:	learn: 0.9204491	test: 0.9187534	best: 0.9189860 (21)	total: 30.2s	remaining: 20m 2s
98:	learn: 0.9204615	test: 0.9188360	best: 0.9189860 (21)	total: 30.6s	remaining: 20m 4s
99:	learn: 0.9205230	test: 0.9188120	best: 0.9189860 (21)	total: 30.8s	remaining: 19m 59s
100:	learn: 0.9205138	test: 0.9188520	best: 0.9189860 (21)	total: 31s	remaining: 19m 55s
101:	learn: 0.9204726	test: 0.9186633	best: 0.9189860 (21)	total: 31.2s	remaining: 19m 50s
102:	learn: 0.9204516	test: 0.9186644	best: 0.9189860 (21)	total: 31.4s	remaining: 19m 46s
103:	learn: 0

183:	learn: 0.9221922	test: 0.9189101	best: 0.9190944 (161)	total: 51.3s	remaining: 17m 43s
184:	learn: 0.9222170	test: 0.9189101	best: 0.9190944 (161)	total: 51.5s	remaining: 17m 42s
185:	learn: 0.9222332	test: 0.9187663	best: 0.9190944 (161)	total: 51.7s	remaining: 17m 40s
186:	learn: 0.9221831	test: 0.9188367	best: 0.9190944 (161)	total: 52s	remaining: 17m 39s
187:	learn: 0.9222164	test: 0.9188627	best: 0.9190944 (161)	total: 52.2s	remaining: 17m 38s
188:	learn: 0.9222612	test: 0.9189239	best: 0.9190944 (161)	total: 52.4s	remaining: 17m 37s
189:	learn: 0.9222476	test: 0.9189239	best: 0.9190944 (161)	total: 52.7s	remaining: 17m 35s
190:	learn: 0.9222474	test: 0.9188649	best: 0.9190944 (161)	total: 52.9s	remaining: 17m 34s
191:	learn: 0.9223473	test: 0.9189827	best: 0.9190944 (161)	total: 53.1s	remaining: 17m 32s
192:	learn: 0.9222563	test: 0.9190405	best: 0.9190944 (161)	total: 53.3s	remaining: 17m 31s
193:	learn: 0.9222833	test: 0.9191109	best: 0.9191109 (193)	total: 53.5s	remaining

273:	learn: 0.9240492	test: 0.9195575	best: 0.9195575 (273)	total: 1m 16s	remaining: 17m 17s
274:	learn: 0.9241493	test: 0.9195575	best: 0.9195575 (274)	total: 1m 16s	remaining: 17m 16s
275:	learn: 0.9242192	test: 0.9195575	best: 0.9195575 (274)	total: 1m 16s	remaining: 17m 18s
276:	learn: 0.9242145	test: 0.9196111	best: 0.9196111 (276)	total: 1m 17s	remaining: 17m 18s
277:	learn: 0.9242341	test: 0.9196111	best: 0.9196111 (276)	total: 1m 17s	remaining: 17m 17s
278:	learn: 0.9242128	test: 0.9195383	best: 0.9196111 (276)	total: 1m 17s	remaining: 17m 17s
279:	learn: 0.9242820	test: 0.9195007	best: 0.9196111 (276)	total: 1m 18s	remaining: 17m 16s
280:	learn: 0.9243368	test: 0.9193341	best: 0.9196111 (276)	total: 1m 18s	remaining: 17m 15s
281:	learn: 0.9243245	test: 0.9192845	best: 0.9196111 (276)	total: 1m 18s	remaining: 17m 21s
282:	learn: 0.9243867	test: 0.9192939	best: 0.9196111 (276)	total: 1m 19s	remaining: 17m 20s
283:	learn: 0.9243981	test: 0.9192563	best: 0.9196111 (276)	total: 1m 

362:	learn: 0.9266772	test: 0.9195142	best: 0.9196601 (359)	total: 1m 41s	remaining: 16m 53s
363:	learn: 0.9267086	test: 0.9194836	best: 0.9196601 (359)	total: 1m 41s	remaining: 16m 52s
364:	learn: 0.9267375	test: 0.9196742	best: 0.9196742 (364)	total: 1m 41s	remaining: 16m 52s
365:	learn: 0.9267395	test: 0.9196130	best: 0.9196742 (364)	total: 1m 42s	remaining: 16m 52s
366:	learn: 0.9267277	test: 0.9195942	best: 0.9196742 (364)	total: 1m 42s	remaining: 16m 52s
367:	learn: 0.9267559	test: 0.9195682	best: 0.9196742 (364)	total: 1m 42s	remaining: 16m 51s
368:	learn: 0.9268090	test: 0.9195988	best: 0.9196742 (364)	total: 1m 42s	remaining: 16m 50s
369:	learn: 0.9268159	test: 0.9196386	best: 0.9196742 (364)	total: 1m 42s	remaining: 16m 49s
370:	learn: 0.9268321	test: 0.9196998	best: 0.9196998 (370)	total: 1m 43s	remaining: 16m 47s
371:	learn: 0.9268447	test: 0.9197610	best: 0.9197610 (371)	total: 1m 43s	remaining: 16m 46s
372:	learn: 0.9269225	test: 0.9197396	best: 0.9197610 (371)	total: 1m 

...

3628:	learn: 0.9550005	test: 0.9204739	best: 0.9212486 (3141)	total: 13m 45s	remaining: 1m 24s
3629:	learn: 0.9550030	test: 0.9205370	best: 0.9212486 (3141)	total: 13m 45s	remaining: 1m 24s
3630:	learn: 0.9549996	test: 0.9205182	best: 0.9212486 (3141)	total: 13m 45s	remaining: 1m 23s
3631:	learn: 0.9549936	test: 0.9204968	best: 0.9212486 (3141)	total: 13m 45s	remaining: 1m 23s
3632:	learn: 0.9550096	test: 0.9205052	best: 0.9212486 (3141)	total: 13m 45s	remaining: 1m 23s
3633:	learn: 0.9550036	test: 0.9205052	best: 0.9212486 (3141)	total: 13m 46s	remaining: 1m 23s
3634:	learn: 0.9549992	test: 0.9205572	best: 0.9212486 (3141)	total: 13m 46s	remaining: 1m 22s
3635:	learn: 0.9550052	test: 0.9204042	best: 0.9212486 (3141)	total: 13m 46s	remaining: 1m 22s
3636:	learn: 0.9550295	test: 0.9203170	best: 0.9212486 (3141)	total: 13m 46s	remaining: 1m 22s
3637:	learn: 0.9550431	test: 0.9203170	best: 0.9212486 (3141)	total: 13m 47s	remaining: 1m 22s
3638:	learn: 0.9550412	test: 0.9203782	best: 0.921

3716:	learn: 0.9552170	test: 0.9204449	best: 0.9212486 (3141)	total: 14m 4s	remaining: 1m 4s
3717:	learn: 0.9552170	test: 0.9204357	best: 0.9212486 (3141)	total: 14m 4s	remaining: 1m 4s
3718:	learn: 0.9552268	test: 0.9203837	best: 0.9212486 (3141)	total: 14m 4s	remaining: 1m 3s
3719:	learn: 0.9552230	test: 0.9204120	best: 0.9212486 (3141)	total: 14m 4s	remaining: 1m 3s
3720:	learn: 0.9552110	test: 0.9203706	best: 0.9212486 (3141)	total: 14m 4s	remaining: 1m 3s
3721:	learn: 0.9552248	test: 0.9203706	best: 0.9212486 (3141)	total: 14m 5s	remaining: 1m 3s
3722:	learn: 0.9552308	test: 0.9203706	best: 0.9212486 (3141)	total: 14m 5s	remaining: 1m 2s
3723:	learn: 0.9552428	test: 0.9204227	best: 0.9212486 (3141)	total: 14m 5s	remaining: 1m 2s
3724:	learn: 0.9552432	test: 0.9203915	best: 0.9212486 (3141)	total: 14m 5s	remaining: 1m 2s
3725:	learn: 0.9552372	test: 0.9203409	best: 0.9212486 (3141)	total: 14m 5s	remaining: 1m 2s
3726:	learn: 0.9552412	test: 0.9204437	best: 0.9212486 (3141)	total: 1

3804:	learn: 0.9553753	test: 0.9203109	best: 0.9212486 (3141)	total: 14m 22s	remaining: 44.2s
3805:	learn: 0.9553693	test: 0.9203109	best: 0.9212486 (3141)	total: 14m 22s	remaining: 44s
3806:	learn: 0.9553816	test: 0.9203109	best: 0.9212486 (3141)	total: 14m 22s	remaining: 43.7s
3807:	learn: 0.9553756	test: 0.9202881	best: 0.9212486 (3141)	total: 14m 22s	remaining: 43.5s
3808:	learn: 0.9553800	test: 0.9203187	best: 0.9212486 (3141)	total: 14m 23s	remaining: 43.3s
3809:	learn: 0.9553740	test: 0.9202667	best: 0.9212486 (3141)	total: 14m 23s	remaining: 43.1s
3810:	learn: 0.9553718	test: 0.9202556	best: 0.9212486 (3141)	total: 14m 23s	remaining: 42.8s
3811:	learn: 0.9553838	test: 0.9202556	best: 0.9212486 (3141)	total: 14m 23s	remaining: 42.6s
3812:	learn: 0.9553838	test: 0.9202342	best: 0.9212486 (3141)	total: 14m 23s	remaining: 42.4s
3813:	learn: 0.9553920	test: 0.9201822	best: 0.9212486 (3141)	total: 14m 24s	remaining: 42.1s
3814:	learn: 0.9553920	test: 0.9202478	best: 0.9212486 (3141)	

3892:	learn: 0.9554994	test: 0.9201295	best: 0.9212486 (3141)	total: 14m 42s	remaining: 24.2s
3893:	learn: 0.9554994	test: 0.9200247	best: 0.9212486 (3141)	total: 14m 42s	remaining: 24s
3894:	learn: 0.9555114	test: 0.9201119	best: 0.9212486 (3141)	total: 14m 42s	remaining: 23.8s
3895:	learn: 0.9555114	test: 0.9201495	best: 0.9212486 (3141)	total: 14m 42s	remaining: 23.6s
3896:	learn: 0.9555114	test: 0.9201211	best: 0.9212486 (3141)	total: 14m 42s	remaining: 23.3s
3897:	learn: 0.9555114	test: 0.9201211	best: 0.9212486 (3141)	total: 14m 43s	remaining: 23.1s
3898:	learn: 0.9555092	test: 0.9200951	best: 0.9212486 (3141)	total: 14m 43s	remaining: 22.9s
3899:	learn: 0.9555092	test: 0.9201327	best: 0.9212486 (3141)	total: 14m 43s	remaining: 22.7s
3900:	learn: 0.9555201	test: 0.9200715	best: 0.9212486 (3141)	total: 14m 43s	remaining: 22.4s
3901:	learn: 0.9555212	test: 0.9200195	best: 0.9212486 (3141)	total: 14m 43s	remaining: 22.2s
3902:	learn: 0.9555212	test: 0.9200807	best: 0.9212486 (3141)	

3980:	learn: 0.9556613	test: 0.9201869	best: 0.9212486 (3141)	total: 15m 19s	remaining: 4.39s
3981:	learn: 0.9556613	test: 0.9201257	best: 0.9212486 (3141)	total: 15m 20s	remaining: 4.16s
3982:	learn: 0.9556613	test: 0.9200905	best: 0.9212486 (3141)	total: 15m 20s	remaining: 3.93s
3983:	learn: 0.9556552	test: 0.9200905	best: 0.9212486 (3141)	total: 15m 21s	remaining: 3.7s
3984:	learn: 0.9556552	test: 0.9201257	best: 0.9212486 (3141)	total: 15m 21s	remaining: 3.47s
3985:	learn: 0.9556543	test: 0.9201869	best: 0.9212486 (3141)	total: 15m 22s	remaining: 3.24s
3986:	learn: 0.9556664	test: 0.9201869	best: 0.9212486 (3141)	total: 15m 22s	remaining: 3.01s
3987:	learn: 0.9556613	test: 0.9201869	best: 0.9212486 (3141)	total: 15m 23s	remaining: 2.78s
3988:	learn: 0.9556638	test: 0.9201869	best: 0.9212486 (3141)	total: 15m 24s	remaining: 2.55s
3989:	learn: 0.9556809	test: 0.9202481	best: 0.9212486 (3141)	total: 15m 24s	remaining: 2.32s
3990:	learn: 0.9556749	test: 0.9201869	best: 0.9212486 (3141)

In [18]:
res

(0.9056944985394824, 0.13857839230804925)

In [17]:
# rank_model.save_model('rank_model_cat_good', format="cbm", export_parameters=None)

In [11]:
rank_model.load_model(fname='rank_model_cat_good')

In [12]:
y_pred = training.predict_with_stack_model(rank_model, model, test, emb, use_y=True)

/Users/sergmiller/Documents/code/python/alissa/generator.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`
  output=model.get_layer(model.layers[5].name).output)


AttributeError: 'CatBoostRegressor' object has no attribute 'predict_proba'

In [17]:
private.shape

(104834, 6)

In [ ]:
private_pred = training.predict_with_stack_model(rank_model, model, private, emb)

In [ ]:
sub = wonder.create_sub(private, private_pred, 'submissions/sub_private_pfound_9212_2level_model.csv')

### classifier

In [65]:
from sklearn.linear_model import LogisticRegression

rank_model = LogisticRegression(max_iter=1)

In [66]:
res, rank_model = generator.train_classifier(rank_model, \
                                model, emb, train, boost, \
                                model_file='rank_model_clf', use_cache=True)

Datasets are loaded
(13899,) (13899,)


In [67]:
res

(0.9061700922551106, 0.14079177603214776)

In [71]:
y_pred = training.predict_with_stack_model(rank_model, model, test, 
                                           emb, use_y=True, use_proba=True)

/Users/sergmiller/Documents/code/python/alissa/generator.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`
  output=model.get_layer(model.layers[5].name).output)


ndcg_mean: 0.8981278469945023, ndcg_std: 0.14531597493836068
